In [ ]:
#!pip install git+https://github.com/rabernat/xmitgcm.git@fix-llcreader-klevels-bug

In [ ]:
import numpy as np
import xmitgcm.llcreader as llcreader
%matplotlib inline
import holoviews as hv
#from holoviews.operation.datashader import regrid
#hv.extension('bokeh')
import xarray as xr
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,10)

%matplotlib inline
import holoviews as hv
from holoviews.operation.datashader import regrid
hv.extension('bokeh')

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)


In [ ]:
model = llcreader.ECCOPortalLLC2160Model()
model

In [ ]:
ds_sst = model.get_dataset(varnames=['Theta'], k_levels=[0], type='latlon')
ds_sst

In [ ]:
#create a daily climatology and anomaly
climatology_mean = ds_sst.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
ds_anomaly = ds_sst.groupby('time.dayofyear')-climatology_mean  #take out annual mean to remove trends
anom = ds_anomaly.Theta.isel(k=0,time=6200,j=slice(4900,5400),i=slice(4600,5200)).load()

In [ ]:
xx = np.arange(600)/600*110 
xlabel = [20,40,60,80,100]
yy = np.arange(500)/500*25+5
ylabel = [10,20,30]
plt.pcolormesh(xx,yy,anom,vmin=-2,vmax=2,cmap='seismic')
plt.tick_params(labelsize=30)
plt.xlabel('Spatial Scale (km)',size=30)
plt.ylabel('Random Uncertainty (%)',size=30)

plt.savefig('discovered_country_anomaly_fullres.png')

In [ ]:
#plt.savefig('discovered_country_anomaly_fullres.png')

In [ ]:
noise = 10
sst = anom
#create random array 0 to 1
tem1a = (sst/sst)*np.random.rand(500,600)
#tem1a.plot()
#create linearly increasing random array
tem2 = (sst/sst).load()
for j in range(500):
    tem2[j,:]=tem2[j,:]*((j/500)**3*noise)
noise = tem1a*tem2
sstnoise = sst+sst*noise

In [ ]:
# okay, now in x direction smooth by radius, convolution ?
anomsmooth=sstnoise.copy(deep=True).load()
noisesmooth=sstnoise.copy(deep=True).load()
for i in range(600):
    w = int(i/20)+1 #window for smoothing
    i1,i2=i-w+1,i+w
    if i1<0:
        i1=0
    if i2>599:
        i2=599
    for j in range(500):
        j1,j2=j-w+1,j+w
        if j1<0:
            j1=0
        if j2>499:
            j2=499
        anomsmooth[j,i]=sst[j1:j2,i1:i2].mean()
        noisesmooth[j,i]=noise[j1:j2,i1:i2].mean()
    if int(i/100)*100==i:
        print(i,600)
anomsmooth.to_netcdf('anomsmooth45a.nc')
noisesmooth.to_netcdf('anomsmooth45a.nc')
plt.rcParams['figure.figsize'] = (20,10)
#(anomsmooth+noisesmooth*anomsmooth).plot(vmin=-2,vmax=2,cmap='seismic')

xx = np.arange(600)/600*110 
xlabel = [20,40,60,80,100]
yy = np.arange(500)/500*25+5
ylabel = [10,20,30]
plt.pcolormesh(xx,yy,anomsmooth+noisesmooth*anomsmooth,vmin=-2,vmax=2,cmap='seismic')
plt.tick_params(labelsize=30)
plt.xlabel('Spatial Scale (km)',size=30)
plt.ylabel('Random Uncertainty (%)',size=30)


#(anomsmooth).plot(vmin=-2,vmax=2,cmap='seismic')
plt.savefig('discovered_country_anomaly_smoothed.png')

In [ ]:
# okay, now in x direction smooth by radius, convolution ?
noise = 1
sst = ds_sst.Theta.isel(k=0,time=6200,j=slice(4900,5400),i=slice(4600,5200)).load()
#create random array 0 to 1
tem1a = (sst/sst)*np.random.rand(500,600)
#tem1a.plot()
#create linearly increasing random array
tem2 = (sst/sst).load()
for j in range(500):
    tem2[j,:]=tem2[j,:]*((j/500)**3*noise)
noise = tem1a*tem2
sstnoise = sst+sst*noise
anomsmooth=sstnoise.copy(deep=True).load()
noisesmooth=sstnoise.copy(deep=True).load()
for i in range(600):
    w = int(i/20)+1 #window for smoothing
    i1,i2=i-w+1,i+w
    if i1<0:
        i1=0
    if i2>599:
        i2=599
    for j in range(500):
        j1,j2=j-w+1,j+w
        if j1<0:
            j1=0
        if j2>499:
            j2=499
        anomsmooth[j,i]=sst[j1:j2,i1:i2].mean()
        noisesmooth[j,i]=noise[j1:j2,i1:i2].mean()
    if int(i/100)*100==i:
        print(i,600)
anomsmooth.to_netcdf('anomsmooth45a.nc')
noisesmooth.to_netcdf('anomsmooth45a.nc')

In [ ]:
plt.rcParams['figure.figsize'] = (18,10)
#(anomsmooth+noisesmooth*anomsmooth).plot(vmin=10,vmax=20,cmap='turbo')
#(anomsmooth).plot(vmin=-2,vmax=2,cmap='seismic')

xx = np.arange(600)/600*110 
xlabel = [20,40,60,80,100]
yy = np.arange(500)/500*25+5
ylabel = [10,20,30]
plt.pcolormesh(xx,yy,anomsmooth+noisesmooth*anomsmooth,vmin=8,vmax=22,cmap='turbo')
plt.tick_params(labelsize=30)
plt.xlabel('Spatial Scale (km)',size=30)
plt.ylabel('Random Uncertainty (%)',size=30)


plt.savefig('discovered_country_sst_smoothed.png')

In [ ]:
plt.rcParams['figure.figsize'] = (18,10)
#(anomsmooth+noisesmooth*anomsmooth).plot(vmin=10,vmax=20,cmap='turbo')
#(anomsmooth).plot(vmin=-2,vmax=2,cmap='seismic')

xx = np.arange(600)/600*260 
xlabel = [20,60,100,140,180,220]
yy = np.arange(500)/500*30+5
ylabel = [10,20,30]
dotx = [35,250,50,62,80,200,20]
doty = [17,18,32,22,25,18,12]
plt.pcolormesh(xx,yy,anomsmooth+noisesmooth*anomsmooth,vmin=8,vmax=22,cmap='turbo')
plt.scatter(dotx,doty)
plt.tick_params(labelsize=30)
plt.xlabel('Spatial Scale (km)',size=30)
plt.ylabel('Random Uncertainty (%)',size=30)


plt.savefig('discovered_country_sst_smoothed_A.png')

In [ ]:
plt.rcParams['figure.figsize'] = (18,10)
#(anomsmooth+noisesmooth*anomsmooth).plot(vmin=10,vmax=20,cmap='turbo')
#(anomsmooth).plot(vmin=-2,vmax=2,cmap='seismic')

xx = np.arange(600)/600*110 
xlabel = [20,60,100]
yy = np.arange(500)/500*30+5
ylabel = [10,20,30]
dotx = [35,50,62,80,20]
doty = [17,32,22,25,12]
plt.pcolormesh(xx,yy,anomsmooth+noisesmooth*anomsmooth,vmin=8,vmax=22,cmap='turbo')
plt.scatter(dotx,doty)
plt.tick_params(labelsize=30)
plt.xlabel('Spatial Scale (km)',size=30)
plt.ylabel('Random Uncertainty (%)',size=30)


plt.savefig('discovered_country_sst_smoothed_B.png')

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)

#anomsmooth.plot(vmin=6,vmax=24,cmap='turbo')
xx = np.arange(600)/600*110 
xlabel = [20,40,60,80,100]
yy = np.arange(500)/500*25+5
ylabel = [10,20,30]
plt.pcolormesh(xx,yy,anomsmooth,vmin=6,vmax=24,cmap='turbo')
plt.tick_params(labelsize=30)
plt.xlabel('Spatial Scale (km)',size=30)
plt.ylabel('Random Uncertainty (%)',size=30)

#(anomsmooth).plot(vmin=-2,vmax=2,cmap='seismic')
plt.savefig('discovered_country_sst_fullres.png')

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
ds_sst.Theta.isel(k=0,time=6200,j=slice(4700,5400),i=slice(4600,5200)).plot(vmin=6,vmax=24,cmap='turbo')


In [ ]:
# apply linearly increasing noise on y axis to simulate how errors increase
# to increase noise increase number j is over.  /1400 is === 0.5 max noise increase

In [ ]:
noise = 0.2
sst = ds_sst.Theta.isel(k=0,time=6200,j=slice(4700,5400),i=slice(4600,5200))
#create random array 0 to 1
tem1 = ds_sst.Theta.isel(k=0,time=6200,j=slice(4700,5400),i=slice(4600,5200))/ds_sst.Theta.isel(k=0,time=6200,j=slice(4700,5400),i=slice(4600,5200))
tem1a = tem1*np.random.rand(700,600)
#tem1a.plot()
#create linearly increasing random array
tem2 = ds_sst.Theta.isel(k=0,time=6200,j=slice(4700,5400),i=slice(4600,5200))/ds_sst.Theta.isel(k=0,time=6200,j=slice(4700,5400),i=slice(4600,5200))
tem2 = tem2.load()
for j in range(700):
    tem2[j,:]=tem2[j,:]*((j/700)*noise)
tem2a = tem1a*tem2
sstnoise = sst+sst*tem2a

In [ ]:
#plot with added noise
plt.rcParams['figure.figsize'] = (20,10)
sstnoise.plot(vmin=6,vmax=24,cmap='turbo')

In [ ]:
%%time
#play around withe w/N to figure out smoothing
xx=sstnoise[0,:].XC
xres=sstnoise[0,:].XC.copy(deep=True)
xx = (xx-xx[0])*np.cos(sstnoise.YC[300,0]*np.pi/180.)*111
for i in range(600):
    w = int(i/35)+1 #window for smoothing
    xres[i] = (xx[200+w]-xx[200-w+1]).data
    if int(i/100)*100==i:
        print(i,600)
print(i,w,xres[i].data)

In [ ]:
# okay, now in x direction smooth by radius, convolution ?
sstsmooth=sstnoise.copy(deep=True).load()
for i in range(600):
    w = int(i/35)+1 #window for smoothing
    i1,i2=i-w+1,i+w
    if i1<0:
        i1=0
    if i2>600:
        i2=600
    for j in range(700):
        j1,j2=j-w+1,j+w
        if j1<0:
            j1=0
        if j2>700:
            j2=700
        sstsmooth[j,i]=sstnoise[j1:j2,i1:i2].mean()
    if int(i/100)*100==i:
        print(i,600)
sstsmooth.to_netcdf('sstsmooth45a.nc')

In [ ]:
sstsmooth = xr.open_dataset('sstsmooth45a.nc')
sstsmooth = sstsmooth.Theta

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
(sstsmooth-sstsmooth.mean()).plot(vmin=-2,vmax=2,cmap='seismic')
#sstsmooth.Theta.plot(vmin=6,vmax=24,cmap='turbo')
plt.savefig('discovered_country_.2_125_a.png')

In [ ]:
sstsmooth=sstnoise.copy(deep=True).load()
for i in range(600):
    w = int(i/45)+1 #window for smoothing
    t1=sstnoise.rolling(j=w,center=True).mean()    
    t2=sstnoise.rolling(i=w,center=True).mean()
    sstsmooth[:,i]=(t1[:,i]+t2[:,i])/2

In [ ]:
#plot with added noise
plt.rcParams['figure.figsize'] = (20,10)
sstsmooth4.plot(vmin=6,vmax=24,cmap='turbo')
plt.savefig('discovered_country4.png')

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
ds_sst.Theta.isel(k=0,time=6200,j=slice(4700,5400),i=slice(4600,5200)).plot(vmin=6,vmax=24,cmap='turbo')
#plt.savefig('theta_global.png')